<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Embeddings/Fairy_tales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import gensim
import numpy as np
import os
import zipfile

from IPython.display import clear_output
from gensim.models import KeyedVectors
from sklearn.linear_model import LinearRegression

###Download vocabularies and vector representation

In [6]:
def download_vec(name):
  path = os.path.dirname(name)
  with zipfile.ZipFile(name, 'r') as zf:
    zf.extractall(path)
  os.remove(name)

In [7]:
!wget -O ukrainian.zip "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/9CAeNsJiInoyUA"
!wget -O russian.zip "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/3yG0-M4M8fypeQ"
clear_output()

for name in ['ukrainian.zip', 'russian.zip']:
  download_vec(name)

--2023-07-05 19:08:50--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/9CAeNsJiInoyUA
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/3a70bb47f6c0f3044a9267856b44f2eb15086a14d873a08fd0d62e302537915a/64a5f430/HdZOcL9j2UCaFZl90Y8xfcevwXkXuXvaIYjUX06MZoTXMyIOD6p5K-p8gA1R1iDaMnWY-AL8LMuEB1uPSIvHDw%3D%3D?uid=0&filename=cc.uk.300.vec.zip&disposition=attachment&hash=0MriIZAL2DprNGr89hL7a6bICrn4LuFQAJY%2BK4qSU1A%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=78404243&fsize=386324207&hid=d843208373e1ea4be77ac5cfc29ca557&media_type=compressed&tknv=v2 [following]
--2023-07-05 19:08:51--  https://downloader.disk.yandex.ru/disk/3a70bb47f6c0f3044a9267856b44f2eb15086a14d873a08fd0d62e302537915a/64a5f430/HdZOcL9j2UCaFZl90Y8xfcevwXkXuXvaIYjUX06MZoTXMyIOD6p5K-p8gA1R1iDaMnWY-

In [19]:
!wget ukr_rus.train.txt https://raw.githubusercontent.com/yandexdataschool/nlp_course/2022/week01_embeddings/ukr_rus.train.txt
!wget ukr_rus.test.txt https://raw.githubusercontent.com/yandexdataschool/nlp_course/2022/week01_embeddings/ukr_rus.test.txt
clear_output()

### Load embeddings

In [21]:
uavec = 'cc.uk.300.vec'
ruvec = 'cc.ru.300.vec'

In [10]:
ua_emb = KeyedVectors.load_word2vec_format(uavec)
ru_emb = KeyedVectors.load_word2vec_format(ruvec)

### Check for two most contradictive words from both languages

In [13]:
ua_emb.most_similar(ua_emb[['паляниця']], topn=10)

[('паляниця', 1.0),
 ('хлібина', 0.6972880959510803),
 ('паляницю', 0.6888859272003174),
 ('паляниці', 0.6719595789909363),
 ('Паляниця', 0.6607965230941772),
 ('пашниця', 0.6127423644065857),
 ('ґаздиня', 0.6104057431221008),
 ('сосонка', 0.6035154461860657),
 ('здоба', 0.6022063493728638),
 ('мисочка', 0.6010351181030273)]

In [15]:
ru_emb.most_similar(ru_emb[['Сыктывкар']], topn=10)

[('Сыктывкар', 1.0000001192092896),
 ('Ухта', 0.7353183031082153),
 ('Усинск', 0.7093490958213806),
 ('Сосногорск', 0.6975598931312561),
 ('Петрозаводск', 0.6828557252883911),
 ('Воркута', 0.6743683815002441),
 ('Сургут', 0.6718339323997498),
 ('Сыктывкара', 0.6710988879203796),
 ('Нижневартовск', 0.6590413451194763),
 ('Йошкар-Ола', 0.6539392471313477)]

"Сыктывкар" is so brutal it's cosine similarity with itself is more than 1.

Thing is there is a lowercase "сыктывкар" with cos.similarity of 0.9999998807.. and if we sum them up and divide by 2 we'll get 1.

### Sample some small vocab pairs

In [23]:
def load_word_pairs(filename, ua_emb=ua_emb, ru_emb=ru_emb):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as vocab:
        for line in vocab:
            ua, ru = line.rstrip().split("\t")
            if ua not in ua_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((ua, ru))
            uk_vectors.append(ua_emb[ua])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [24]:
ua_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
ua_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")